In [ ]:
import openai
import pandas as pd
import spacy
nlp = spacy.load("en_core_web_lg")
import statistics
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import numpy as np
import collections
from collections import Counter
import re
import language_tool_python  
from nltk.tokenize import sent_tokenize
from gingerit.gingerit import GingerIt
import pprint
from spacy import displacy
from sklearn.svm import SVC
import textstat
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.metrics import BigramAssocMeasures
import math
from nltk.sentiment import SentimentIntensityAnalyzer

In [9]:
def add_feature(df, column_index, feature):
    # Get the column names
    columns = df.columns.tolist()

    # Insert the new feature at the specified index
    columns.insert(column_index, feature)

    # Reindex the dataframe with the updated column order
    df = df.reindex(columns=columns)

    return df

def walk_tree1(node, depth):
    if node.n_lefts + node.n_rights > 0:
        dep=[]
        d=0
        for child in node.children:
            d=walk_tree1(child, depth + 1)
            dep.append(d)
        return max(dep)
    
    else:
        return depth
    
def avg_sentence_complexity(str1):
    final_list=[]
    nlp = spacy.load("en_core_web_lg")
    temp_list=[]
    sents=[]
    txt = str1
    txt=" ".join(txt.split())
    text = nlp(txt)
    sent_tokenize1 = (list(text.sents))
    sents.append(sent_tokenize1[0])
    for sent in text.sents:
        d=walk_tree1(sent.root, 0)
        temp_list.append(d)
            #displacy.render(sent, style="dep")
        final_list.append(temp_list)
        #print("essay first line ",sents[len(sents)-1],"max is ",max(temp_list)," min is ",min(temp_list)," mean is ", round(statistics.mean(temp_list),3),
        #  " median is ",statistics.median(temp_list)," mode is ",statistics.mode(temp_list))
    return round(statistics.mean(temp_list),3)

def grammar_check(str1):

    my_tool = language_tool_python.LanguageTool('en-GB')###########
    matches = my_tool.check(str1)
    list_s1=[]
    if(matches):
        fs=str1.split(".")[0]
        #print(i," index | essay first sentences ",fs)
        for match in matches:
            print(match)
    #print(match.ruleId)
    #print('Message:',match.message,"\n")
            #print("####################### ",match.context)
            if match.replacements:
                chopped_string = match.replacements[0].replace(',','').strip()
                list_s1.append(chopped_string)
            else:
                list_s1.append(match.ruleId)    
        #print("final list",list_s1)
    return len(list_s1)

def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    list1=[" ".join(ngram) for ngram in ngrams]
    #return [" ".join(ngram) for ngram in ngrams]
    return list1

def rep_count(str1):
    tot_rp=[]
    max_rp=[]
    #print(ess[i],"\n","new","\n")
    list1=generate_ngrams(str1, 5)
    myset = set(list1)
    rept = len(list1) - len(myset)
    tot_rp.append(rept)
    c = collections.Counter(list1)
    c.most_common(1)[0][1]
    max_rp.append(c.most_common(1)[0][1])
    #print("\n\nnew essay index ",i,": for essay ",list1[0],":")
    print("repeat ",rept," times | most common phrase is **",c.most_common(1)[0][0], "** |number of repeataion happened: ",c.most_common(1)[0][1]," times\n")
    #print("\ncommon phrases: \n",c)
    #print("\nmy set: \n",myset)
    #print("list ",tot_rp," ",max_rp)
    #print("individual ",rept," ",c.most_common(1)[0][1])
    return rept,c.most_common(1)[0][1],c.most_common(10)

def most_frequent(str1):

    nlp = spacy.load("en_core_web_lg")
    str1=" ".join(str1.split())
    text = nlp(str1)
    words = [token.text for token in text if token.is_stop!= True and token.is_punct!= True]
    w_c=Counter(words)
    m_f=w_c.most_common(1)[0][1]
    
    tags=[]
    for w in text:
        tags.append(w.pos_)
    tag_freq = Counter(tags)
    #print ("=====tag FREQUENCY=====")
    #print(tag_freq.values())

    w_l=word_cnt(str1)
    for key in tag_freq:
        #print("hi")
        if key=='NOUN':
            n=tag_freq[key]/w_l
            #print("n ",n)
        elif key == 'VERB':
            v=tag_freq[key]/w_l
            #print("v ",v)
        elif key =='ADJ':
            a=tag_freq[key]/w_l
            #print("a ",a)
        elif key =='PRON':
            p=tag_freq[key]/w_l

        elif key=='MD':
            print("the number of modal verbs ",key, tag_freq.values())
            #print("p ",p)
        #print("key is ",key," value ",tag_freq[key], " frequency ", tag_freq[key]/w_l)
    return m_f,n,v,a,p

def uniq_w(str1):
    return len(set(str1.split()))

def stop_w(str1):
    nlp = spacy.load("en_core_web_lg")
    text = nlp(str1)
    words = [token.text for token in text if token.is_stop == True]
    #print(words)
    return(len(words))

def stop_w_r(str1):
    number_of_sentences = sent_tokenize(str1)
    n=len(number_of_sentences)

    nlp = spacy.load("en_core_web_lg")
    text = nlp(str1)
    words = [token.text for token in text if token.is_stop == True]
    #print(words)
    
    l=len(words)
    swr=l/n
    return swr

def sen_len(str1):
    number_of_sentences = sent_tokenize(str1)
    i=0
    avg=0
    sl=[]
    for num in number_of_sentences:
        i+=1
        words=num.split()
        avg+=len(words)
        sl.append(len(words))
    #print(num, len(num),len(words))
    avg=avg/i
    md=statistics.mode(sl)
    mx=max(sl)
    mn=min(sl)
    return avg,md,mx,mn

def sen_num(str1):
    number_of_sentences = sent_tokenize(str1)
    n=len(number_of_sentences)
    #print(n)
    return n

def word_cnt(str1):
    words=str1.split()
    return len(words)

def lexical_diversity(str1):
    tokens = word_tokenize(str1)

    lex_div = len(set(tokens)) / len(tokens)
    print("Lexical Diversity Score of essay:", lex_div)
    return lex_div

def discourse_conn(str1):
    tokens = nltk.word_tokenize(str1)

    # Perform part-of-speech tagging
    pos_tags = nltk.pos_tag(tokens)

    # Identify discourse connectors based on part-of-speech patterns
    discourse_connectors = [token for token, pos in pos_tags if pos in ['CC', 'RB', 'IN']]

    # Output the identified discourse connectors
    print("Discourse connectors:", discourse_connectors)
    print("Discourse connector count:", len(discourse_connectors))
    return len(discourse_connectors)

def sentence_similarity(str1):
    text = nlp(str1)
    sent_tokenize = (list(text.sents))
    i=0
    sensim=0.0
    sensim_list=[]
    for sent2 in sent_tokenize:
        i+=1
        if i == 1:
            sent1=sent2
            continue
        #print (sent2.similarity(sent1))
        sensim+=sent2.similarity(sent1)
        sensim_score=sent2.similarity(sent1)
        sent1=sent2
        sensim_list.append(sensim_score)
    sensim=sensim/i
    #print("average sentence similarity is ",sensim)

    return sensim,sensim_list

def word_similarity(str1):
    text = nlp(str1)
    p=0
    ps=0
    n=0
    ns=0
    total=0
    word = [token.text for token in text if token.is_punct != True]
    for i in range(len(word)):
    #x.append(i)
    #print(i)
    #print(words[i])
        w2=word[i]
        if i == 0:
            w1=w2
            continue
    #print (nlp(w2).similarity(nlp(w1)))
        #print(w1+ " "+w2+" ",nlp(w2).similarity(nlp(w1)))
        if nlp(w2).similarity(nlp(w1))>0:
            p=p+1
            ps+=nlp(w2).similarity(nlp(w1))
        elif nlp(w2).similarity(nlp(w1))<0:
            n=n+1
            ns+=nlp(w2).similarity(nlp(w1))
    #y1=y1+float(nlp(w2).similarity(nlp(w1)))
    #print(y1)
    #y.append(y1)
        total+=nlp(w2).similarity(nlp(w1))
        w1=w2
    #print(p,n)
    return p,n,total,ps,ns

def get_modal_verbs(text):
    #nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    cnt=0
    modal_verbs = []
    for token in doc:
        if token.pos_ == 'VERB' and token.tag_ == 'MD':
            modal_verbs.append(token.lemma_)
            cnt+=1

    return cnt,modal_verbs

def calculate_readability(text):
    # Calculate the Flesch-Kincaid Grade Level
    flesch_grade = textstat.flesch_kincaid_grade(text)

    # Calculate the Automated Readability Index (ARI)
    ari = textstat.automated_readability_index(text)

    # Calculate the Coleman-Liau Index
    coleman_liau = textstat.coleman_liau_index(text)

    # Return the readability scores
    return flesch_grade, ari, coleman_liau

def calculate_cohesion(text):
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and punctuation
    stop_words = set(stopwords.words("english"))
    tokens = [token.lower() for token in tokens if token.isalnum() and token.lower() not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Convert generator to a list of bigrams
    bigrams = list(nltk.bigrams(tokens))

    # Calculate bigram frequencies
    bigram_freq = Counter(bigrams)

    # Calculate pointwise mutual information (PMI) for bigrams
    total_bigrams = len(bigrams)
    pmi_scores = {}
    for bigram, freq in bigram_freq.items():
        pmi = math.log(freq / total_bigrams / ((tokens.count(bigram[0]) / len(tokens)) * (tokens.count(bigram[1]) / len(tokens))))
        pmi_scores[bigram] = pmi

    # Calculate average PMI as a measure of cohesion
    avg_pmi = sum(pmi_scores.values()) / len(pmi_scores)

    return avg_pmi

def calculate_uniqueness_score(text):
    # Tokenize the text into words
    words = word_tokenize(text)

    # Calculate the frequency distribution of words
    word_freq = Counter(words)

    # Calculate the uniqueness score as the ratio of unique words to total words
    uniqueness_score = len(word_freq) / len(words)

    return uniqueness_score

def detect_emotional_tone(text):
    sid = SentimentIntensityAnalyzer()
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores

def para_cnt(str1):
    paragraphs = str1.split("\n\n")
    return len(paragraphs)

def discourse_marker():
    with open('../other/da.txt',"r") as file:
        text=file.read()
    d_list = text.split(",")
    #print(d_list)
    d_list.remove('')
    #index = d_list.index('')
    #print(index)
    #d_list.remove('')
    return d_list

def discourse_marker_check(essay):
    d_list=discourse_marker()
    count=0
    for marker in d_list:
        occurrences = re.findall(r'\b' + re.escape(marker) + r'\b', essay, flags=re.IGNORECASE)
        count += len(occurrences)
        #print("occurances ",occurrences," for marker ",marker)
    #print("count ",count)
    return count

In [3]:
df = pd.DataFrame(columns=['essay','topic'])
df.to_csv('../csv/human/csv_human-1.csv',index=False)

In [ ]:
import glob
df = pd.DataFrame()
for path in glob.glob('../human/input/*.txt'):   
    with open(path) as file:
        first_line = file.readline().strip()
        remaining_content = file.read()
        if remaining_content.startswith('\n'):
            remaining_content = remaining_content[1:]
    df=df.append({'essay':remaining_content,'topic':first_line},ignore_index=True)
    '''ch=input("enter e")
    if ch =='e' or ch =='E':
        break
    else:
        continue'''

In [ ]:
pd.set_option('display.max_columns', None)
print(df)

In [7]:
df.to_csv('../csv/human/csv_human-1.csv',mode='a',header=None,index=False)

read from csv

In [8]:
data = pd.read_csv('../csv/human/csv_human-1.csv')
ess=data.loc[:,'essay']

In [ ]:
data

or read from df

In [53]:
data=df
data
ess=data.loc[:,'essay']

paragraph count

In [10]:
for i in range(len(ess)):
    pc=para_cnt(ess[i])
    data.loc[i,'para_count']=pc

word count

In [11]:
for i in range(len(ess)):
    wc=word_cnt(ess[i])
    data.loc[i,'word_count']=wc

sentence count

In [12]:
for i in range(len(ess)):
    ns=sen_num(ess[i])
    data.loc[i,'sentence_count']=ns

sentence length

In [13]:
for i in range(len(ess)):
    avg,md,mx,mn=sen_len(ess[i])
    data.loc[i,'sentence_len(mean)']=avg
    data.loc[i,'sentence_len(mode)']=md
    data.loc[i,'sentence_len(max)']=mx
    data.loc[i,'sentence_len(min)']=mn

stopwords

In [14]:
for i in range(len(ess)):
    sw=stop_w(ess[i])
    swr=stop_w_r(ess[i])
    data.loc[i,'stop_words']=sw
    data.loc[i,'sw_rates']=swr

In [ ]:
data.to_csv('../csv/human/csv_human-1.csv',index=False)
data = pd.read_csv('../csv/human/csv_human-1.csv')
ess=data.loc[:,'essay']
pd.set_option('display.max_columns', None)
print(data)

unique words

In [16]:
for i in range(len(ess)):
    uw=uniq_w(ess[i])
    data.loc[i,'unique_words']=uw

pos and most frequent

In [17]:
for i in range(len(ess)):
    uw=uniq_w(ess[i])
    mf,n,v,a,p=most_frequent(ess[i])
    mvc,mv=get_modal_verbs(ess[i])
    data.loc[i,'noun']=n
    data.loc[i,'verb']=v
    data.loc[i,'adj']=a
    data.loc[i,'pron']=p,
    data.loc[i,'modal verb']=mvc
    data.loc[i,'most_freq']=mf

In [ ]:
data

In [ ]:
data.to_csv('../csv/human/csv_human-1.csv',index=False)
data = pd.read_csv('../csv/human/csv_human-1.csv')
ess=data.loc[:,'essay']
pd.set_option('display.max_columns', None)
print(data)

repeatation of phrases

In [ ]:
for i in range(len(ess)):
    tr,mr,rep_list=rep_count(ess[i])
    data.loc[i,'number_of_repeatation']=tr
    data.loc[i,'max_occurance_of_most_frequent_phrase']=mr


In [ ]:
data

sentence complexity

In [21]:
for i in range(len(ess)):
    asc=avg_sentence_complexity(ess[i])
    data.loc[i,'avg sentence complexity']=asc

In [ ]:
data.to_csv('../csv/human/csv_human-1.csv',index=False)
data = pd.read_csv('../csv/human/csv_human-1.csv')
ess=data.loc[:,'essay']
pd.set_option('display.max_columns', None)
print(data)

lexical diversity

In [ ]:
for i in range(len(ess)):
    l_d=lexical_diversity(ess[i])
    data.loc[i,'lexical diversity']=l_d

sentence similarity

In [24]:
for i in range(len(ess)):
    sensim,sensim_list=sentence_similarity(ess[i])
    sensim_list.sort()
    sensim_mod=statistics.mode(sensim_list)
    sensim_med=statistics.median(sensim_list)
    data.loc[i,'sentence similarity mean']=sensim
    data.loc[i,'sentence similarity median']=sensim_med
    data.loc[i,'sentence similarity mode']=sensim_mod,

word similarity

In [ ]:
for i in range(len(ess)):
    pos,neg,tot,p_s,n_s=word_similarity(ess[i])
    data.loc[i,'word_sim_p_count']=pos
    data.loc[i,'word_sim_p_score']=p_s
    data.loc[i,'word_sim_n_count']=neg
    data.loc[i,'word_sim_n_score']=n_s
    data.loc[i,'word_sim_score']=tot

In [ ]:
data.to_csv('../csv/human/csv_human-1.csv',index=False)
data = pd.read_csv('../csv/human/csv_human-1.csv')
ess=data.loc[:,'essay']
pd.set_option('display.max_columns', None)
print(data)

readability score

In [29]:
for i in range(len(ess)):
    f_g, ari, c_l = calculate_readability(ess[i])
    data.loc[i,'flesch grade']=f_g
    data.loc[i,'Automated Readability']=ari
    data.loc[i,'coleman_liau']=c_l

cohesion score

In [30]:
for i in range(len(ess)):
    coh_score=calculate_cohesion(ess[i])
    data.loc[i,'cohesion score']=coh_score

uniqueness

In [31]:
for i in range(len(ess)):
    uniq = calculate_uniqueness_score(ess[i])
    data.loc[i,'uniqueness of words']=uniq

emotional tone

In [32]:
for i in range(len(ess)):
    emot = detect_emotional_tone(ess[i])
    data.loc[i,'positive_sentiment']=emot['pos']
    data.loc[i,'negative_sentiment']=emot['neg']
    data.loc[i,'neutral_sentiment']=emot['neu']

In [ ]:
data

grammar and spelling 
***GB and US***

In [35]:
import language_tool_python

def grammar_check1(text):
# Create separate LanguageTool instances for GB and US English
    tool_gb = language_tool_python.LanguageTool('en-GB')
    tool_us = language_tool_python.LanguageTool('en-US')


# Check for GB English grammar errors
    matches_gb = tool_gb.check(text)
    print("gb ",matches_gb)

# Check for US English grammar errors
    matches_us = tool_us.check(text)
    print("us ",matches_us)

    common_errors = [
    error for error in matches_gb
    if any(error.matches(match) for match in matches_us)
    ]

    print("Errors common to GB and US English:")
    for error in common_errors:
        print(error)

# Filter errors that do not match in either GB or US English
#    non_matching_errors = [
#        error for error in matches_gb + matches_us
#        if not any(error.matches(match) for match in matches_gb + matches_us)
#    ]

#    print("Errors not matching in either GB or US English:")
#    for error in non_matching_errors:
#        print(error)

In [ ]:
data = pd.read_csv('../csv/human/csv_human-1.csv')
ess=data.loc[:,'essay']
for i in range(len(ess)):
    print("essay ",i," and topic is ",data.loc[i,"topic"])
    g_s=grammar_check1(ess[i])
    #data.loc[i,'grammar&spel']=g_s

discourse annotators

In [ ]:
data = pd.read_csv('../csv/human/csv_human-1.csv')
ess=data.loc[:,'essay']
########### entering new column ###################
new_feature = 'discourse_marker'
column_index = data.shape[1]-1
data = add_feature(data, column_index, new_feature)
pd.set_option('display.max_columns', None)
print("\nAfter adding new feature:")
print(data)

In [14]:
for i in range(len(ess)):
    cnt=discourse_marker_check(ess[i])
    data.loc[i,'discourse_marker']=cnt

In [18]:
data

,essay,topic,para_count,word_count,sentence_count,sentence_len(mean),sentence_len(mode),sentence_len(max),sentence_len(min),stop_words,sw_rates,unique_words,noun,verb,adj,pron,modal verb,most_freq,number_of_repeatation,max_occurance_of_most_frequent_phrase,avg sentence complexity,lexical diversity,sentence similarity mean,sentence similarity median,sentence similarity mode,word_sim_p_count,word_sim_p_score,word_sim_n_count,word_sim_n_score,word_sim_score,flesch grade,Automated Readability,coleman_liau,cohesion score,uniqueness of words,positive_sentiment,negative_sentiment,neutral_sentiment,discourse_marker,label
0,"Recently, a great deal of attention has been p...",Alternative forms of transport to alleviate tr...,5.0,388.0,20.0,19.400000,13.0,38.0,12.0,193.0,9.650000,199.0,0.306701,0.128866,0.097938,0.030928,9.0,11.0,5.0,2.0,5.400,0.444186,0.835664,0.880775,0.799130,380.0,150.254682,3.0,-0.333771,149.920911,12.0,13.9,13.69,4.182405,0.444186,0.100,0.066,0.834,49.0,0.0
1,Gun is definitely not a kind of normal good in...,Gun control,5.0,280.0,16.0,17.500000,19.0,38.0,10.0,143.0,8.937500,178.0,0.296429,0.121429,0.082143,0.025000,9.0,9.0,0.0,1.0,5.188,0.532915,0.849118,0.905018,0.882263,276.0,108.484310,0.0,0.000000,108.484310,8.9,11.1,10.84,4.028763,0.532915,0.053,0.282,0.664,32.0,0.0
2,The increasing population and the technologica...,Government should make more effort to promote ...,4.0,256.0,15.0,17.066667,20.0,24.0,8.0,135.0,9.000000,138.0,0.261719,0.132812,0.148438,0.023438,9.0,8.0,1.0,2.0,5.667,0.480287,0.844501,0.912760,0.817024,254.0,107.812128,0.0,0.000000,107.812128,11.1,11.5,11.83,3.702329,0.480287,0.173,0.026,0.801,28.0,0.0
3,"Technology, in the past two hundred years, has...",technology that widen the gap between rich and...,6.0,380.0,18.0,21.111111,27.0,38.0,10.0,202.0,11.222222,223.0,0.257895,0.105263,0.110526,0.050000,5.0,8.0,0.0,1.0,5.778,0.486047,0.864111,0.920834,0.860589,375.0,160.342955,0.0,0.000000,160.342955,11.5,12.8,11.08,4.393480,0.486047,0.198,0.038,0.764,47.0,0.0
4,"For most folks on Earth, the need to gather in...",Sporting events easing international tensions,4.0,372.0,15.0,24.800000,28.0,42.0,14.0,177.0,11.800000,227.0,0.244624,0.131720,0.112903,0.021505,6.0,10.0,0.0,1.0,5.867,0.544118,0.864177,0.935544,0.844644,363.0,146.918488,4.0,-0.445256,146.473232,11.8,15.9,13.18,4.533942,0.544118,0.160,0.053,0.786,55.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,"In this world, there is no one who does not kn...",Giving aid to poorer countries has more negati...,6.0,381.0,18.0,21.166667,20.0,34.0,7.0,189.0,10.500000,228.0,0.265092,0.139108,0.162730,0.031496,6.0,15.0,0.0,1.0,5.778,0.523474,0.850632,0.909514,0.859106,376.0,151.923564,0.0,0.000000,151.923564,11.6,14.1,12.71,4.280784,0.523474,0.160,0.124,0.716,49.0,0.0
86,"In present day and age, students are normally ...",Benefits of students doing extra jobs,5.0,346.0,18.0,19.222222,16.0,28.0,11.0,171.0,9.500000,204.0,0.274566,0.158960,0.098266,0.034682,8.0,10.0,0.0,1.0,5.944,0.509044,0.870809,0.922498,0.876775,341.0,138.879977,2.0,-0.171047,138.708930,12.0,14.2,14.10,4.160972,0.509044,0.194,0.046,0.760,54.0,0.0
87,Preserving wild animals in zoos is an issue wh...,zoos should be built to protect rural animals,4.0,310.0,13.0,23.846154,19.0,47.0,12.0,157.0,12.076923,169.0,0.251613,0.148387,0.122581,0.032258,12.0,11.0,0.0,1.0,6.357,0.483680,0.853539,0.928610,0.867761,306.0,124.190676,0.0,0.000000,124.190676,12.6,14.0,11.49,3.940267,0.483680,0.144,0.090,0.765,50.0,0.0
88,"Today, we no longer fear any contagious diseas...",Those who use drugs should be automatically se...,4.0,433.0,21.0,20.619048,15.0,53.0,6.0,257.0,12.238095,236.0,0.210162,0.157044,0.076212,0.071594,12.0,7.0,0.0,1.0,5.571,0.472803,0.871059,0.909387,0.859901,427.0,189.225096,1.0,-0.014288,189.210807,10.1,11.0,9.34,4.489194,0.472803,0.127,0.121,0.752,62.0,0.0


In [16]:
data.to_csv('../csv/human/csv_human-1.csv',index=False)

In [17]:
data

,essay,topic,para_count,word_count,sentence_count,sentence_len(mean),sentence_len(mode),sentence_len(max),sentence_len(min),stop_words,sw_rates,unique_words,noun,verb,adj,pron,modal verb,most_freq,number_of_repeatation,max_occurance_of_most_frequent_phrase,avg sentence complexity,lexical diversity,sentence similarity mean,sentence similarity median,sentence similarity mode,word_sim_p_count,word_sim_p_score,word_sim_n_count,word_sim_n_score,word_sim_score,flesch grade,Automated Readability,coleman_liau,cohesion score,uniqueness of words,positive_sentiment,negative_sentiment,neutral_sentiment,discourse_marker,label
0,"Recently, a great deal of attention has been p...",Alternative forms of transport to alleviate tr...,5.0,388.0,20.0,19.400000,13.0,38.0,12.0,193.0,9.650000,199.0,0.306701,0.128866,0.097938,0.030928,9.0,11.0,5.0,2.0,5.400,0.444186,0.835664,0.880775,0.799130,380.0,150.254682,3.0,-0.333771,149.920911,12.0,13.9,13.69,4.182405,0.444186,0.100,0.066,0.834,49.0,0.0
1,Gun is definitely not a kind of normal good in...,Gun control,5.0,280.0,16.0,17.500000,19.0,38.0,10.0,143.0,8.937500,178.0,0.296429,0.121429,0.082143,0.025000,9.0,9.0,0.0,1.0,5.188,0.532915,0.849118,0.905018,0.882263,276.0,108.484310,0.0,0.000000,108.484310,8.9,11.1,10.84,4.028763,0.532915,0.053,0.282,0.664,32.0,0.0
2,The increasing population and the technologica...,Government should make more effort to promote ...,4.0,256.0,15.0,17.066667,20.0,24.0,8.0,135.0,9.000000,138.0,0.261719,0.132812,0.148438,0.023438,9.0,8.0,1.0,2.0,5.667,0.480287,0.844501,0.912760,0.817024,254.0,107.812128,0.0,0.000000,107.812128,11.1,11.5,11.83,3.702329,0.480287,0.173,0.026,0.801,28.0,0.0
3,"Technology, in the past two hundred years, has...",technology that widen the gap between rich and...,6.0,380.0,18.0,21.111111,27.0,38.0,10.0,202.0,11.222222,223.0,0.257895,0.105263,0.110526,0.050000,5.0,8.0,0.0,1.0,5.778,0.486047,0.864111,0.920834,0.860589,375.0,160.342955,0.0,0.000000,160.342955,11.5,12.8,11.08,4.393480,0.486047,0.198,0.038,0.764,47.0,0.0
4,"For most folks on Earth, the need to gather in...",Sporting events easing international tensions,4.0,372.0,15.0,24.800000,28.0,42.0,14.0,177.0,11.800000,227.0,0.244624,0.131720,0.112903,0.021505,6.0,10.0,0.0,1.0,5.867,0.544118,0.864177,0.935544,0.844644,363.0,146.918488,4.0,-0.445256,146.473232,11.8,15.9,13.18,4.533942,0.544118,0.160,0.053,0.786,55.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,"In this world, there is no one who does not kn...",Giving aid to poorer countries has more negati...,6.0,381.0,18.0,21.166667,20.0,34.0,7.0,189.0,10.500000,228.0,0.265092,0.139108,0.162730,0.031496,6.0,15.0,0.0,1.0,5.778,0.523474,0.850632,0.909514,0.859106,376.0,151.923564,0.0,0.000000,151.923564,11.6,14.1,12.71,4.280784,0.523474,0.160,0.124,0.716,49.0,0.0
86,"In present day and age, students are normally ...",Benefits of students doing extra jobs,5.0,346.0,18.0,19.222222,16.0,28.0,11.0,171.0,9.500000,204.0,0.274566,0.158960,0.098266,0.034682,8.0,10.0,0.0,1.0,5.944,0.509044,0.870809,0.922498,0.876775,341.0,138.879977,2.0,-0.171047,138.708930,12.0,14.2,14.10,4.160972,0.509044,0.194,0.046,0.760,54.0,0.0
87,Preserving wild animals in zoos is an issue wh...,zoos should be built to protect rural animals,4.0,310.0,13.0,23.846154,19.0,47.0,12.0,157.0,12.076923,169.0,0.251613,0.148387,0.122581,0.032258,12.0,11.0,0.0,1.0,6.357,0.483680,0.853539,0.928610,0.867761,306.0,124.190676,0.0,0.000000,124.190676,12.6,14.0,11.49,3.940267,0.483680,0.144,0.090,0.765,50.0,0.0
88,"Today, we no longer fear any contagious diseas...",Those who use drugs should be automatically se...,4.0,433.0,21.0,20.619048,15.0,53.0,6.0,257.0,12.238095,236.0,0.210162,0.157044,0.076212,0.071594,12.0,7.0,0.0,1.0,5.571,0.472803,0.871059,0.909387,0.859901,427.0,189.225096,1.0,-0.014288,189.210807,10.1,11.0,9.34,4.489194,0.472803,0.127,0.121,0.752,62.0,0.0


In [4]:
print(f"The number of columns in the CSV file is: {data.shape[1]}")

The number of columns in the CSV file is: 39


essay                                    Computer-a device which has given a whole new ...
topic                                                                       computers..use
para_count                                                                             5.0
word_count                                                                           377.0
sentence_count                                                                        28.0
sentence_len(mean)                                                               13.464286
sentence_len(mode)                                                                    17.0
sentence_len(max)                                                                     23.0
sentence_len(min)                                                                      5.0
stop_words                                                                           188.0
sw_rates                                                                          6.714286

label

In [38]:
for i in range(len(ess)):
    data.loc[i,'label']=0

In [ ]:
data.to_csv('../csv/human/csv_human-1.csv',index=False)
data = pd.read_csv('../csv/human/csv_human-1.csv')
ess=data.loc[:,'essay']
pd.set_option('display.max_columns', None)
print(data)

In [ ]:
data

check duplicate topics

In [ ]:
seen = set()
duplicates = []
    
for item in topics:
        if item in seen:
            if item not in duplicates:
                duplicates.append(item)
        else:
            seen.add(item)
print(duplicates)